<h1 align="center"><font color="yellow">Advanced RAG using Llama Index</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Baseado no tutorial de `Plaban Nayak`

![](https://miro.medium.com/v2/resize:fit:786/format:webp/1*DPWC__Fls59QNakGYagjIQ.png)

# <font color="red">Contextualizando</font>

<font color="orange">Aqui implementaremos um conceito para melhorar a recuperação (`retrieval`) que pode ser útil para o processamento de texto com reconhecimento de conteúdo, onde também consideraríamos o contexto circundante de uma frase para compreender insights valiosos.</font>

# <font color="red">O que é Llama-Index</font>

<font color="orange">LlamaIndex é um Framework de dados para aplicativos baseados em `LLM` para `ingerir`, `estruturar` e `acessar dados privados` ou `específicos de domínio`.</font>

# <font color="red">Como usar o Llama-Index?</font>

O uso básico é um processo de cinco etapas que nos leva de nossos `dados brutos (raw)` e `não estruturados` ao conteúdo gerado pelo LLM com base nesses dados.

* Carregar documentos

* Parse documentos em Nodes

* Build Index

* Query o índice

* Parse a resposta

In [ ]:
# Instale as dependências necessárias:

%pip install llama-index -qU
%pip install -q openai
%pip install pypdf
%pip install doc2txt
%pip install -qU llama-cpp-python
%pip install transformers
%pip install accelerate

In [1]:
# Importar dependências necessárias:

import os
import openai
from getpass import getpass
#
import logging
import sys
from pprint import pprint
#
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
#
from llama_index import(VectorStoreIndex,
                        SimpleDirectoryReader,
                        load_index_from_storage,
                        StorageContext,
                        ServiceContext,
                        Document)

from llama_index.llms import OpenAI,HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.text_splitter import SentenceSplitter
from llama_index.embeddings import OpenAIEmbedding,HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from llama_index.postprocessor import MetadataReplacementPostProcessor

# <font color="red">O que é `Document`?</font>

<font color="orange">O `Document` é um contêiner que contém dados de várias fontes, como `PDF`, uma saída de API ou dados recuperados de um banco de dados.</font>

In [16]:
documents = SimpleDirectoryReader('./Data/').load_data()
print(len(documents))
pprint(documents)

5
[Document(id_='05d8bc33-dc0d-420d-b329-f6cca7e3b52a', embedding=None, metadata={'page_label': '1', 'file_name': 'spacy_teste.pdf', 'file_path': 'Data/spacy_teste.pdf', 'file_type': 'application/pdf', 'file_size': 91986, 'creation_date': '2024-01-08', 'last_modified_date': '2024-01-08', 'last_accessed_date': '2024-01-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='b14de57b59c4f1dbafb1561783c5c64f9babf75c28e9e62c4dab508301d31efd', text='CHATGPT: Granular ity clust ering\nQ: Que significa a granular idade de um clust ering\nA granularidade em um clustering refere-se ao nív el de detalhament o da divisão  \ndos dados em gr upos ou clust ers. \nPor exemplo, em um conjunto de dados de vendas de produtos em uma loja,  \nA escolha da granularidade depend

<font color="orange">O `PDF` foi convertido em uma lista de 5 elementos.</font>

In [17]:
documents[0].get_content()

'CHATGPT: Granular ity clust ering\nQ: Que significa a granular idade de um clust ering\nA granularidade em um clustering refere-se ao nív el de detalhament o da divisão  \ndos dados em gr upos ou clust ers. \nPor exemplo, em um conjunto de dados de vendas de produtos em uma loja,  \nA escolha da granularidade depende do objetivo do clustering e da interpretação  \ndos resultados. Em alguns casos,  uma granular idade alta pode ajudar a  \nidentificar p adrões e caract erísticas específicas dos dados , enquanto em outros  \ncasos, uma granular idade b aixa pode ser suficient e para identificar t endências  \ngerais e insights.\nQ: Podes fazer ex emplos do conceit o\nClaro, aqui estão alguns exemplos para ilustrar o conceito de granularidade em  \nclustering:\nExemplo 1: Clust ering de compras online\nImagine que você esteja trabalhando em uma empresa de comércio eletrônico e  \nqueira agrupar os clientes com base em suas compras.  \nExemplo 2: Clust ering de dados de saúde\nSuponha que 

In [20]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'spacy_teste.pdf',
 'file_path': 'Data/spacy_teste.pdf',
 'file_type': 'application/pdf',
 'file_size': 91986,
 'creation_date': '2024-01-08',
 'last_modified_date': '2024-01-08',
 'last_accessed_date': '2024-01-08'}

In [21]:
documents[1].metadata

{'page_label': '2',
 'file_name': 'spacy_teste.pdf',
 'file_path': 'Data/spacy_teste.pdf',
 'file_type': 'application/pdf',
 'file_size': 91986,
 'creation_date': '2024-01-08',
 'last_modified_date': '2024-01-08',
 'last_accessed_date': '2024-01-08'}

In [22]:
# Novo valor que adicionamos:
Chave_spaCy = 'valor_spaCy'

# Adicionando o novo valor a todos os documentos:
for document in documents:
    document.metadata['Entidade_spaCy'] = Chave_spaCy

# Verificando os metadados atualizados:
for document in documents:
    print(document.metadata)


{'page_label': '1', 'file_name': 'spacy_teste.pdf', 'file_path': 'Data/spacy_teste.pdf', 'file_type': 'application/pdf', 'file_size': 91986, 'creation_date': '2024-01-08', 'last_modified_date': '2024-01-08', 'last_accessed_date': '2024-01-08', 'Entidade_spaCy': 'valor_spaCy'}
{'page_label': '2', 'file_name': 'spacy_teste.pdf', 'file_path': 'Data/spacy_teste.pdf', 'file_type': 'application/pdf', 'file_size': 91986, 'creation_date': '2024-01-08', 'last_modified_date': '2024-01-08', 'last_accessed_date': '2024-01-08', 'Entidade_spaCy': 'valor_spaCy'}
{'page_label': '3', 'file_name': 'spacy_teste.pdf', 'file_path': 'Data/spacy_teste.pdf', 'file_type': 'application/pdf', 'file_size': 91986, 'creation_date': '2024-01-08', 'last_modified_date': '2024-01-08', 'last_accessed_date': '2024-01-08', 'Entidade_spaCy': 'valor_spaCy'}
{'page_label': '4', 'file_name': 'spacy_teste.pdf', 'file_path': 'Data/spacy_teste.pdf', 'file_type': 'application/pdf', 'file_size': 91986, 'creation_date': '2024-01-08

In [23]:
# Verificando para o primeiro elemento:

documents[0].metadata

{'page_label': '1',
 'file_name': 'spacy_teste.pdf',
 'file_path': 'Data/spacy_teste.pdf',
 'file_type': 'application/pdf',
 'file_size': 91986,
 'creation_date': '2024-01-08',
 'last_modified_date': '2024-01-08',
 'last_accessed_date': '2024-01-08',
 'Entidade_spaCy': 'valor_spaCy'}

In [24]:
# Verificando para o segundo elemento:

documents[1].metadata

{'page_label': '2',
 'file_name': 'spacy_teste.pdf',
 'file_path': 'Data/spacy_teste.pdf',
 'file_type': 'application/pdf',
 'file_size': 91986,
 'creation_date': '2024-01-08',
 'last_modified_date': '2024-01-08',
 'last_accessed_date': '2024-01-08',
 'Entidade_spaCy': 'valor_spaCy'}

# <font color="red">Configurando `llm`</font>

In [ ]:
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    #query_wrapper_prompt=PromptTemplate("<|system|>Please check if the following pieces of context has any mention of the keywords provided in the question.If not ten say that you do not know the answer.Please do not make up your own answer.</s>\n<|user|>\nQuestion:{query_str}</s>\n<|assistant|>\n"),
    # query_wrapper_prompt=PromptTemplate(template),
    context_window=4096,
    max_new_tokens=512,
    model_kwargs={'trust_remote_code':True},
    generate_kwargs={"temperature": 0.0},
    device_map="auto",)


# <font color="red">Configurando o modelo de `Embedding`</font>

In [ ]:
from llama_index.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")